In [ ]:
knitr::opts_chunk$set(cache = TRUE)



# Introduction

AzureML provides an interface to publish web services on Microsoft Azure Machine Learning (Azure ML) from your local R environment. The main functions in the package cover the following topics:

* Workspace: connect to and manage AzureML workspaces
* Datasets: upload and download datasets to and from AzureML workspaces
* Publish: publish R functions as AzureML web services, and update or delete existing services
* Consume: apply any AzureML web service to your R data

## Installing the AzureML Package

This short tutorial will show you how to use the `AzureML` package to operationalize R models in Azure as webservices. Our package version is 0.2.13, which you can install from an up-to-date CRAN mirror. Please be careful, as package version < 0.2 will not work as described here, as there were some fundamental changes in the latter releases.



In [ ]:
getOption("repos")
# on centOS you'll need xml
# system("sudo yum install libxml2-devel")
install.packages("AzureML")
packageVersion("AzureML")


## Getting Started

Getting started with the AzureML package requires a Microsoft Azure Machine Learning account. These are available for free from [studio.azureml.net](https://studio.azureml.net/). Please make an account, don't use a guest account.

A complete introduction on using the `AzureML` package is provided by the package's [vignette](https://cran.r-project.org/web/packages/AzureML/vignettes/getting_started.html).

Firstly, we need to get our AzureML credentials. Using the left-hand menu, click on the _Settings_ tab and copy your _Workspace ID_.

Next, click on the _Authorization Tokens_ tab at the top of the settings page and copy your _Primary Authentication_ key.

In order to connect your R client to your AzureML workshop, we will need to define a workspace object using the `workspace` function inside of your R session that points to an Azure service.



In [ ]:
wsid <- "your_id"
pri_token <- "your_private_token"

library(AzureML)
ws <- workspace(
  id = wsid,
  auth = pri_token
)



In order to use `AzureML` package we need to expose our credentials to the package. For example, we could make this pointer directly using the `workspace` function:



In [ ]:
library(AzureML)
ws <- workspace(
  id = "your workspace ID",
  auth = "your authorization token"
)



Alternatively, you could save your credentials in a _json_ file in your home directory under `~/.azureml/settings.json`.

## Example Usage

The simplest way to use the `AzureML` package is to query your AzureML environment



In [ ]:
head(datasets(ws))


Using `AzureML`, you can download and upload datasets from and to Azure. Examples of such use is provided in the vignette, and isn't of great interest for us today.

## Creating Experiments

We will use the `AzureML` package to create experiments in AzureML.



In [ ]:
e <- experiments(ws)
head(e)


The `experiments` function creates an R `data.frame` containing all the experiments you have in your AzureML account, along with some useful metadata about each experiment.

The same information is also available inside the `ws` object under the slot `experiments`, which is continuously updated whenever called:



In [ ]:
identical(ws$experiments, e)


## Publishing New Webservices

Using the `publishWebService` function, you can publish any R function into an `AzureML` webservice.

Here's a very contrived and silly where we create a webservice for calculating powers based on two scalar arguments.

Firt we create the function:



In [ ]:
powers <- function(x, y) {
  x^y
}


And then we create our webservice:



In [ ]:
api <- publishWebService(
  ws,
  fun = powers,
  name = "azure-power-ex",
  inputSchema = list(
    x = "numeric",
    y = "numeric"
  ),
  outputSchema = list(
    ans = "numeric"
  )
)



## Consuming Webservices

The vignette goes into deeper detail on how to update webservices, or discover them using the package's API. We will instead focus on consuming webservices directly through the package API:




In [ ]:

df <- data.frame(
  x = rep(2, 10),
  y = 1:10
)
s <- services(ws, name = "azure-power-ex")
s <- tail(s, 1) # use the last published function, in case of duplicate function names
ep <- endpoints(ws, s)
consume(ep, df)



## Publishing MRS Models

In the third module of our course, we learned how to create MRS using the `RevoScaleR` package.

In order to deploy them on Azure, we need to first convert them into an object that Azure can understand. At the current time, Azure can only publish webservices using open-source R, so we need to convert our MRS model objects into their open-source counterparts. Fortunately, every MRS model object has a method that'll convert it to an open-source variant with the same coefficeints.

For the purpose of this part of the tutorial, I will assume you have ran the third module and created the gradient boosted trees model for default prediction, `default_model_gbm`.



In [ ]:
default_model_gbm <- readRDS("gbm.rds")
summary(default_model_gbm)


To convert it to to a `gbm` object from the base `gbm` package, we will use the obvious method: `as.gbm`:



In [ ]:
gbm_default <- as.gbm(default_model_gbm)
summary(gbm_default)


We will use the first five rows of the test dataset as our sample data:



In [ ]:
mort_split <- readRDS("mort_split.rds")
test <- head(mort_split$validate)

mypredict <- function(newdata)
{
  require(gbm)
  predict(gbm_default, newdata, n.trees = 10)
}

print(mypredict(test))



Now that we have create a scoring wrapper function, let's publish it to Azure:



In [ ]:
ep <- publishWebService(ws = ws,
                        fun = mypredict,
                        name = "AzureML-rpart-ex",
                        inputSchema = test)

# Consume test data, comparing with result above
print(consume(ep, test))


## Deleting a Webservice

If you want to delete a webservice, simply specify the webservice name and use the `deleteWebService` function:



In [ ]:
deleteWebService(ws, name = "azure-power-ex")
deleteWebService(ws, name = "AzureML-rpart-ex")